In [1]:
import pyspark
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

In [3]:
import os
os.environ["JAVA_HOME"] = 'C:\Program Files\Java\jre-1.8'

In [4]:
spark = SparkSession.builder.appName('pillsdata').getOrCreate()

In [5]:
spark

In [10]:
DEA_data = spark.read.csv('D:/opioids/arcos_all_washpost/arcos_all_washpost.tsv', sep='\t',header=True)

In [11]:
DEA_data.show()

+---------------+----------------+--------------------+---------------------+-----------------+-----------------+-------------+--------------+------------+---------------+------------+-------------------+--------------------+--------------------+--------------------+---------------+----------+-----------+---------+------------+----------------+---------+-----------+-----------+--------+----+----------------+-------------+-------------+--------+----------------+------------------+-----------+--------------+--------------------+--------------------+-------+---------------------+---------------------+--------------------+--------------------+-------+
|REPORTER_DEA_NO|REPORTER_BUS_ACT|       REPORTER_NAME|REPORTER_ADDL_CO_INFO|REPORTER_ADDRESS1|REPORTER_ADDRESS2|REPORTER_CITY|REPORTER_STATE|REPORTER_ZIP|REPORTER_COUNTY|BUYER_DEA_NO|      BUYER_BUS_ACT|          BUYER_NAME|  BUYER_ADDL_CO_INFO|      BUYER_ADDRESS1| BUYER_ADDRESS2|BUYER_CITY|BUYER_STATE|BUYER_ZIP|BUYER_COUNTY|TRANSACTION_COD

In [7]:
import pyspark.sql.functions as f
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

DEA_data = DEA_data.select(['BUYER_STATE','BUYER_COUNTY','DRUG_NAME','DOSAGE_UNIT','TRANSACTION_DATE'])

DEA_data = DEA_data.withColumn(
    "year",
    F.substring(F.col("TRANSACTION_DATE"), 5, 4)
)

DEA_data = DEA_data.withColumn(
    "month",
    F.substring(F.col("TRANSACTION_DATE"), 1, 2)
)

DEA_data = DEA_data.drop('TRANSACTION_DATE')

#DEA_data = DEA_data.withColumn("year",DEA_data.year.cast(IntegerType()))
#DEA_data = DEA_data.withColumn("month",DEA_data.year.cast(IntegerType()))
#DEA_data.filter((DEA_data.year==2006)&(DEA_data.month==1))

DEA_hyd = DEA_data.filter(DEA_data.DRUG_NAME == 'HYDROCODONE') 
DEA_hyd = DEA_hyd.drop('DRUG_NAME')
#DEA_hyd = DEA_hyd.withColumnRenamed("QUANTITY", "hydQUANTITY")
DEA_hyd = DEA_hyd.withColumn("DOSAGE_UNIT",DEA_hyd.DOSAGE_UNIT.cast(IntegerType()))
DEA_hyd = DEA_hyd.groupBy('BUYER_STATE','BUYER_COUNTY','year','month').sum('DOSAGE_UNIT')

DEA_hyd.toPandas().to_csv('D:/opioids/hydrocodone_agg.csv')

In [8]:
DEA_oxy = DEA_data.filter(DEA_data.DRUG_NAME == 'OXYCODONE')
DEA_oxy = DEA_oxy.drop('DRUG_NAME')
#DEA_oxy = DEA_oxy.withColumnRenamed("QUANTITY", "oxyQUANTITY")
DEA_oxy = DEA_oxy.withColumn("DOSAGE_UNIT",DEA_oxy.DOSAGE_UNIT.cast(IntegerType()))

DEA_oxy = DEA_oxy.groupBy('BUYER_STATE','BUYER_COUNTY','year','month').sum('DOSAGE_UNIT')

DEA_oxy.toPandas().to_csv('D:/opioids/oxycodone_agg.csv')


In [6]:
pop_data = spark.read.text('D:/opioids/us.1969_2020.19ages.adjusted.txt')

In [7]:
pop_data.show()

+--------------------+
|               value|
+--------------------+
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991910...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991911...|
|1969AL01001991920...|
+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

pop_data = pop_data.withColumn(
    "year",
    F.substring(F.col("value"), 1, 4)
)

pop_data = pop_data.withColumn(
    "state",
    F.substring(F.col("value"), 5, 2)
)

pop_data = pop_data.withColumn(
    "state_fip",
    F.substring(F.col("value"), 7, 2)
)

pop_data = pop_data.withColumn(
    "county_fip",
    F.substring(F.col("value"), 9, 3)
)

pop_data = pop_data.withColumn(
    "registry",
    F.substring(F.col("value"), 12, 2)
)
pop_data = pop_data.withColumn(
    "race",
    F.substring(F.col("value"), 14,1)
)
pop_data = pop_data.withColumn(
    "origin",
    F.substring(F.col("value"), 15, 1)
)

pop_data = pop_data.withColumn(
    "sex",
    F.substring(F.col("value"), 16, 1)
)

pop_data = pop_data.withColumn(
    "age",
    F.substring(F.col("value"), 17, 2)
)

pop_data = pop_data.withColumn(
    "population",
    F.substring(F.col("value"), 19, 8)
)

pop_data = pop_data.drop('value')

pop_data = pop_data.withColumn("population",pop_data.population.cast(IntegerType()))
pop_data = pop_data.withColumn("year",pop_data.year.cast(IntegerType()))

pop_data = pop_data.filter((pop_data.year>2005) & (pop_data.year<2015))

pop_data = pop_data.groupBy('year','state','state_fip','county_fip','registry').sum('population')

pop_data = pop_data.withColumnRenamed("sum(population)", "population")

pop_data.toPandas().to_csv('D:/opioids/population0614.csv')